In [12]:
# ======================================================================
# 📌 1) Import des librairies
# ======================================================================
import os
from roboflow import Roboflow
from pathlib import Path
import torch

print("✔️ Librairies importées")


✔️ Librairies importées


In [13]:
# ======================================================================
# 📌 2) Connexion à Roboflow + téléchargement du dataset
# ======================================================================

# ⚠️ Remplacer par ton API Key personnelle (ne jamais partager publiquement)
RF_API_KEY = "4zGCDGgIjB5Lg8TVVqSY"

rf = Roboflow(api_key=RF_API_KEY)
project = rf.workspace("deep-nhhm8").project("my-first-project-prs1r")
version = project.version(2)

dataset = version.download("yolov9")  # <-- VARIABLE CRÉÉE ICI (corrige ton erreur)

print("📁 Dataset téléchargé dans :", dataset.location)


loading Roboflow workspace...
loading Roboflow project...
📁 Dataset téléchargé dans : /content/My-First-Project-2


In [14]:
# ======================================================================
# 📌 3) Vérification du dataset
# ======================================================================
data_yaml = str(Path(dataset.location) / "data.yaml")

print("✔️ Fichier data.yaml trouvé :", data_yaml)

with open(data_yaml, 'r') as f:
    print("\n===== CONTENU data.yaml =====")
    print(f.read())


✔️ Fichier data.yaml trouvé : /content/My-First-Project-2/data.yaml

===== CONTENU data.yaml =====
names:
- poubelle_pleine
- poubelle_vide
nc: 2
roboflow:
  license: CC BY 4.0
  project: my-first-project-prs1r
  url: https://universe.roboflow.com/deep-nhhm8/my-first-project-prs1r/dataset/2
  version: 2
  workspace: deep-nhhm8
test: ../test/images
train: ../train/images
val: ../valid/images



In [15]:
# ======================================================================
# 📌 4) Entraînement YOLOv9
# ======================================================================
from ultralytics import YOLO

print("📦 Chargement du modèle YOLOv9…")
model = YOLO("yolov9c.pt")  # tu peux changer pour yolov9e, v9s, v9t

print("\n🚀 Début de l'entraînement…")

results = model.train(
    data=data_yaml,
    epochs=50,
    imgsz=640,
    batch=8,
    workers=2,
    device=0 if torch.cuda.is_available() else "cpu"
)

print("🎉 Entraînement terminé !")


📦 Chargement du modèle YOLOv9…

🚀 Début de l'entraînement…
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/My-First-Project-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9c.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, over

In [16]:
# ======================================================================
# 📌 5) Évaluation du modèle
# ======================================================================

metrics = model.val()

print("\n===== METRIQUES =====")
print(f"Précision  (mp)     : {metrics.box.mp:.4f}")
print(f"Rappel     (mr)     : {metrics.box.mr:.4f}")
print(f"mAP@50              : {metrics.box.map50:.4f}")
print(f"mAP@50-95           : {metrics.box.map:.4f}")



Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9c summary (fused): 156 layers, 25,320,790 parameters, 0 gradients, 102.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1445.7±348.1 MB/s, size: 37.2 KB)
val: Scanning /content/My-First-Project-2/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 25/25 52.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 1.5it/s 1.3s
                   all         25         36      0.619      0.611      0.635      0.351
       poubelle_pleine         18         27      0.683      0.667      0.732      0.447
         poubelle_vide          8          9      0.555      0.556      0.539      0.255
Speed: 6.8ms preprocess, 34.9ms inference, 0.1ms loss, 1.8ms postprocess per image
Results saved to /content/runs/detect/val

===== METRIQUES =====
Précision  (mp)     : 0.6189
Rappel     (mr)     : 0.6111
mAP@50   

In [17]:
# ======================================================================
# 📌 6) Inférence sur une image
# ======================================================================

# Exemple : choisir une image test dans /test ou uploader une image
image_dir = dataset.location + "/train/images"
test_image = os.path.join(image_dir, os.listdir(image_dir)[0])

print("🔍 Image test :", test_image)
results = model.predict(test_image, save=True)
results


🔍 Image test : /content/My-First-Project-2/train/images/img_202_jpg.rf.8fc3b092aac5be92e84751481ef65e54.jpg

image 1/1 /content/My-First-Project-2/train/images/img_202_jpg.rf.8fc3b092aac5be92e84751481ef65e54.jpg: 640x640 5 poubelle_vides, 47.9ms
Speed: 2.8ms preprocess, 47.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'poubelle_pleine', 1: 'poubelle_vide'}
 obb: None
 orig_img: array([[[  3,   1,   1],
         [251, 249, 249],
         [252, 250, 250],
         ...,
         [246, 249, 254],
         [247, 250, 255],
         [247, 250, 255]],
 
        [[  3,   1,   1],
         [251, 249, 249],
         [252, 250, 250],
         ...,
         [249, 252, 255],
         [249, 252, 255],
         [249, 252, 255]],
 
        [[  3,   1,   1],
         [250, 248, 248],
         [251, 249, 249],
         ...,
         [249, 252, 255],
         [250, 253, 255],
         [250, 253, 255]],
 
        ...,
 
        [[  2,   3,   1],
         [246, 247, 245],
         [255, 255, 254],
         ...,
         [222, 227, 225],
         [223, 228, 226],
         [224, 229, 227]],
 
        [[  3,   4,   2],
         [249, 250, 248],
         [255, 255, 254],
   

In [18]:
# ======================================================================
# 📌 7) Téléchargement automatique du modèle entraîné
# ======================================================================

best_model_path = "runs/detect/train/weights/best.pt"

if os.path.exists(best_model_path):
    print("✔️ Modèle trouvé :", best_model_path)
else:
    print("❌ ERREUR : best.pt introuvable")



✔️ Modèle trouvé : runs/detect/train/weights/best.pt
